In [1]:
import uuid
import time
import pickle
import sys
import gym.spaces
import itertools
import numpy as np
import random
import tensorflow                as tf
import tensorflow.contrib.layers as layers
from collections import namedtuple
from collections import Counter
from importlib import reload


from mods import multiplayer_tools, TicTacToe,Players, DQN


/Users/christophermiller/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.reset_default_graph()
from collections import Counter

#Define the placeholders
#TODO: update placeholders
observation_placeholder = tf.placeholder(shape = [None,2, 3,3], dtype = tf.int32, name = "obs_placeholder")
action_placeholder = tf.placeholder(shape = [None], dtype = tf.int32, name = "act_placeholder")

#target place holder is r(s,a) + \gamma \max_a Q(s',a)
target_placeholder = tf.placeholder(shape = [None], dtype = tf.float32, name = "target_placeholder")

#Define the model and loss function
model = DQN.TicTacToe_model(observation_placeholder, scope = "Q_learn")
update_op = DQN.symbolic_Q_update(model, target_placeholder, action_placeholder)

#Start a session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#Define the players
player = Players.NN_Player(model, model, sess, observation_placeholder, duplicate = False)
opponent = Players.Random_Player()
judge =  Players.NN_Player(model, model, player.session, observation_placeholder, duplicate = True)


#Define the environment
env = TicTacToe.mnk_game()

#Load current net
temp_file_name = './bot_10_31_q_v2.ckpt'

#Want to duplicate session
#saver = tf.train.Saver()
#saver.restore(sess, temp_file_name)

replay_buffer = []
first_state = None
step = 0
tic = time.time()
total_buffer_time = 0
total_runtime = 0
while True:
    step += 1

    #Collect rollouts
    tic_buffer = time.time()
    paths, _ = multiplayer_tools.batch_rollout(player, opponent, env, max_time_steps = 300)

    #Add rollouts to the replay buffer
    if len(replay_buffer) > 100000:
        replay_buffer = replay_buffer[30000:]
    
    replay_buffer += paths
    #Collect samples from our replay buffer
    states, actions, next_states, rewards, masks, not_end_of_path = DQN.sample_paths(replay_buffer, batch_size = 100)
    total_buffer_time +=  (time.time() - tic_buffer)

    
    tic_runtime = time.time()
    #Compute target values
    target_values = DQN.compute_target_values(judge, next_states, masks, not_end_of_path, rewards, verbose=False)

    #Update the network
    player.session.run(update_op, feed_dict= {observation_placeholder : states, action_placeholder : actions, target_placeholder : target_values })
    total_runtime += (time.time() - tic_runtime)
    
    if step % 10 ==0:
        _, player_batch_winners = multiplayer_tools.batch_rollout(player, opponent, env, max_time_steps=1000)
        _, judge_batch_winners = multiplayer_tools.batch_rollout(judge, opponent, env, max_time_steps=1000)
        frac_player_loss = (player_batch_winners[2]*1.0)/(player_batch_winners[0] + player_batch_winners[1] + player_batch_winners[2])
        frac_judge_loss = (judge_batch_winners[2]*1.0)/(judge_batch_winners[0] + judge_batch_winners[1] + judge_batch_winners[2])
        if frac_player_loss <= frac_judge_loss:
            print("replacing ")
            judge = Players.NN_Player(model, model, player.session, observation_placeholder, duplicate = True)


    #Occasionally, test the model and replace it with a previous iteration
    if step% 2 ==0:
        print("")
        print("")
        print("Time since last update", time.time() - tic)
        print("Total gradient step runtime:",total_runtime)
        print("Total buffer time:", total_buffer_time)
        total_runtime = 0
        total_buffer_time = 0
        tic = time.time()
        step = 0
        
        #Print current board distribution
        initial_board = np.array(sess.run(model, feed_dict= {observation_placeholder : [np.array([[[0,0,0],[0,0,0],[0,0,0]],[[0,0,0],[0,0,0],[0,0,0]]]) == 1] }))
        print(np.reshape(initial_board, newshape = (3,3)))

#        player.epsilon = 0
#        expert_player = Players.Expert_Player()
#        child_player = Players.Child_Player()
#        _, expert_batch_winners = multiplayer_tools.batch_rollout(player, expert_player, env, max_time_steps=500)
#        _, child_batch_winners = multiplayer_tools.batch_rollout(player, child_player, env, max_time_steps=500)
#         print(expert_batch_winners)
#        expert_batch_percentages = np.array([expert_batch_winners[0], expert_batch_winners[1], expert_batch_winners[2]])*1.0/(expert_batch_winners[0] + expert_batch_winners[1] + expert_batch_winners[2])
#        child_batch_percentages = np.array([child_batch_winners[0], child_batch_winners[1], child_batch_winners[2]])*1.0/(child_batch_winners[0] + child_batch_winners[1] + child_batch_winners[2])
#        player.epsilon = .2
#        print("percentages against expert", expert_batch_percentages.tolist())
#        print("percentages against child", child_batch_percentages.tolist())

        print("Size of replay buffer:", len(replay_buffer))
        print("")
        print("")
#         _, player_batch_winners = batch_rollout(player, expert_player, env, max_time_steps=1000)
#         frac_player_loss = (player_batch_winners[2]*1.0)/(player_batch_winners[0] + player_batch_winners[1] + player_batch_winners[2])
#        print(player_batch_winners)
#         _, opponent_batch_winners = batch_rollout(opponent, expert_player, env, max_time_steps=1000)
#         frac_opponent_loss = (opponent_batch_winners[2]*1.0)/(opponent_batch_winners[0] + opponent_batch_winners[1] + opponent_batch_winners[2])
        
        
#         if frac_player_loss <= frac_opponent_loss:
        opponent = Players.NN_Player(model, model, sess, observation_placeholder, duplicate = True)


    

duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt


Time since last update 0.9635438919067383
Total gradient step runtime: 0.2928659915924072
Total buffer time: 0.6700561046600342
[[0.         0.         0.01916552]
 [0.         0.00743866 0.        ]
 [0.00743747 0.01971388 0.01574934]]
Size of replay buffer: 161


duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt


Time since last update 1.0499870777130127
Total gradient step runtime: 0.014540910720825195
Total buffer time: 0.7988376617431641
[[0.         0.01309419 0.03582752]
 [0.01392579 0.01734316 0.00580919]
 [0.02335954 0.03249276 0.02124703]]
Size of replay buffer: 360


duplicating session to freeze weights for evaluation...
INFO:tensorflow:Restoring parameters from ./to_duplicate.ckpt
Destroying NN_Player and Session...


Time since last update 1.095479965209961
Total gradient step runtime: 

KeyboardInterrupt: 